In [1]:
import sys
sys.path.append('src')
from torchsummary import summary
from src.config import Config
from dataclasses import asdict
from src.training.train import train_model
import wandb

# Define residual blocks
residual_blocks = [
    # Two residual block with identity mapping
    {"num_channels":64, "output_channels":64, "is_identity":True, "stride":2},
    {"num_channels":64, "output_channels":64, "is_identity":True, "stride":2},
    # Next 3 [conv mapping + identity mapping]
    {"num_channels":64, "output_channels":128, "is_identity":False, "stride":2},
    {"num_channels":128, "output_channels":128, "is_identity":True, "stride":2},
    {"num_channels":128, "output_channels":256, "is_identity":False, "stride":2},
    {"num_channels":256, "output_channels":256, "is_identity":True, "stride":2},
    {"num_channels":256, "output_channels":512, "is_identity":False, "stride":2},
    {"num_channels":512, "output_channels":512, "is_identity":True, "stride":2}
]

config = Config(residual_blocks, nb_classes=10)

summary(config.model, (3, 32, 32), device=config.device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           9,472
       BatchNorm2d-2           [-1, 64, 14, 14]             128
              ReLU-3           [-1, 64, 14, 14]               0
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Conv2d-5             [-1, 64, 7, 7]          36,928
       BatchNorm2d-6             [-1, 64, 7, 7]             128
              ReLU-7             [-1, 64, 7, 7]               0
            Conv2d-8             [-1, 64, 7, 7]          36,928
       BatchNorm2d-9             [-1, 64, 7, 7]             128
           Conv2d-10             [-1, 64, 4, 4]           4,160
             ReLU-11             [-1, 64, 4, 4]               0
    ResidualBlock-12             [-1, 64, 4, 4]      

In [2]:
# Checking GPU and logging to wandb

!wandb login 38835c5bbc76ab228d493d9c4755d75233187f55

!nvidia-smi

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\nicol/.netrc


Sun Apr  9 12:36:25 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2060       WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P2               27W /  N/A|   2909MiB /  6144MiB |     37%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!wandb online  # online / offline to activate or deactivate WandB logging

wandb_config = asdict(config)
wandb_config['model'] = str(config.model)
wandb_config['residual_blocks'] = [str(block) for block in config.residual_blocks]

with wandb.init(
        config=wandb_config,
        project='INF8225 - Project',
        group='Resnet18',
        save_code=True,
    ):
    train_model(config.model, config)

Usage: wandb online [OPTIONS]
Try 'wandb online --help' for help.

Error: Got unexpected extra arguments (# online / offline to activate or deactivate WandB logging)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nicolasdepelteau (fun-stuff). Use `wandb login --relogin` to force relogin


Starting training for 5 epochs, using cuda.

Epoch 1


loss,▁
training_accuracy,▁
validation_accuracy,▁
loss,2.24836
training_accuracy,0.15625
validation_accuracy,0.1


KeyboardInterrupt: 